# JUICE RPWI HF SID2 (RAW): L1a Mask Formation -- 2024/4/7

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import copy
import csv

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_0-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib  as juice_cdf
import juice_sid2_lib as juice_sid2
# import juice_math_lib as juice_math

# Setting and Read CDF file: set by User

In [ ]:
# *** Plot dump ***
dump_mode = 0                           # 0: no-dump  1:plot dump

# *** CAL set ***
band_mode = 0                           # [Power]      0: sum          1: /Hz
unit_mode = 0                           # [Power]      0: raw          1: dBm@ADC   2: V@HF   3:V2@HF   4:V2@RWI
cal_mode  = 0                           # [Power]      0: background   1: CAL       2: all

# *** Unit set ***
f_mode    = 1                           # [Frequency]  0: linear       1: log
time_mode = 1                           # [Time]       0: Epoch        1: data number

# *** Frenquency range set ***
f_mode_min = 3000;   f_mode_max = 10000

# *** Filter make ***
filter_mode  = 0                        # 0: read       1: make
filter_mode2 = 0                        # 0: floor/ceil 1: normal

# *** Power range set ***
p_raw_max = 7.0                         # background: 7.5   CAL: 10
p_raw_min = 0.0                         # background: 2.5   CAL: 5
cf, p_max0, p_min0 = juice_cdf.cal_factors(unit_mode, p_raw_max, p_raw_min)
print("conversion factor:", cf, "   MAX-min:", p_max0, p_min0)

# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
# *** Flight - Ver.1
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2023/'        # CDF data folder

"""
# *** 20230419 ***
data_name_list = ['04/19/SID2_20230419T135849-20230419T141229.cdf',
                  '04/19/SID2_20230419T141231-20230419T141402.cdf',
                 ]

# *** 20230530 ***
data_name_list = ['05/30/SID2_20230530T100326-20230530T100925.cdf',
                  '05/30/SID2_20230530T100927-20230530T100937.cdf',
                 ]
data_name_list = ['05/30/JUICE_L1a_RPWI-HF-SID2_20230530T100326-20230530T100925_V01___RPR1_52000010_2023.150.10.40.53.663.cdf',
                  '05/30/JUICE_L1a_RPWI-HF-SID2_20230530T100927-20230530T100937_V01___RPR1_52000011_2023.150.10.41.53.508.cdf',
                 ]
                 # *** 20230601 ***
data_name_list = ['06/01/SID2_20230601T120759-20230601T120857.cdf',
                  '06/01/SID2_20230601T121435-20230601T121533.cdf',
                  '06/01/SID2_20230601T122138-20230601T122236.cdf',
                  '06/01/SID2_20230601T122707-20230601T122805.cdf',
                 ]

# *** all: 202305-06 ***
data_name_list = ['05/30/SID2_20230530T100326-20230530T100925.cdf',
                  '05/30/SID2_20230530T100927-20230530T100937.cdf',
                  '06/01/SID2_20230601T120759-20230601T120857.cdf',
                  '06/01/SID2_20230601T121435-20230601T121533.cdf',
                  '06/01/SID2_20230601T122138-20230601T122236.cdf',
                  '06/01/SID2_20230601T122707-20230601T122805.cdf',
                 ]

# *** 20230712-13 ***
data_name_list = ['07/12/SID2_20230712T090434-20230712T093848.cdf',
                  '07/12/SID2_20230712T093942-20230712T101355.cdf',
                  '07/12/SID2_20230712T101449-20230712T104147.cdf',
                 ]
data_name_list = ['07/12/SID2_20230712T104149-20230712T232406.cdf']
#
data_name_list = ['07/12/SID2_20230712T232408-20230712T235156.cdf']
# type-III
data_name_list = ['07/12/SID2_20230712T235158-20230713T001854.cdf',
                  '07/13/SID2_20230713T001856-20230713T004644.cdf',
                 ]
data_name_list = ['07/13/SID2_20230713T023720-20230713T030416.cdf',
                 ]
data_name_list = ['07/12/SID2_20230712T232408-20230712T235156.cdf',
                  '07/12/SID2_20230712T235158-20230713T001854.cdf',
                  '07/13/SID2_20230713T001856-20230713T004644.cdf',
                  '07/13/SID2_20230713T004648-20230713T011342.cdf',
                  '07/13/SID2_20230713T011436-20230713T014134.cdf',
                  '07/13/SID2_20230713T014136-20230713T020924.cdf',
                  '07/13/SID2_20230713T020928-20230713T023718.cdf',
                  '07/13/SID2_20230713T023720-20230713T030416.cdf',
                  '07/13/SID2_20230713T030510-20230713T033208.cdf',
                  '07/13/SID2_20230713T033210-20230713T040000.cdf',
                  '07/13/SID2_20230713T040002-20230713T042751.cdf',
                  '07/13/SID2_20230713T042753-20230713T045449.cdf',
                  '07/13/SID2_20230713T045543-20230713T050917.cdf',
                 ]
"""
# 20230712-13 - all
data_name_list = ['07/12/SID2_20230712T090434-20230712T093848.cdf',
                  '07/12/SID2_20230712T093942-20230712T101355.cdf',
                  '07/12/SID2_20230712T101449-20230712T104147.cdf',
                  '07/12/SID2_20230712T104149-20230712T232406.cdf',
                  '07/12/SID2_20230712T232408-20230712T235156.cdf',
                  '07/12/SID2_20230712T235158-20230713T001854.cdf',
                  '07/13/SID2_20230713T001856-20230713T004644.cdf',
                  '07/13/SID2_20230713T004648-20230713T011342.cdf',
                  '07/13/SID2_20230713T011436-20230713T014134.cdf',
                  '07/13/SID2_20230713T014136-20230713T020924.cdf',
                  '07/13/SID2_20230713T020928-20230713T023718.cdf',
                  '07/13/SID2_20230713T023720-20230713T030416.cdf',
                  '07/13/SID2_20230713T030510-20230713T033208.cdf',
                  '07/13/SID2_20230713T033210-20230713T040000.cdf',
                  '07/13/SID2_20230713T040002-20230713T042751.cdf',
                  '07/13/SID2_20230713T042753-20230713T045449.cdf',
                  '07/13/SID2_20230713T045543-20230713T050917.cdf',
                 ]

"""
# 2023 ALL
data_name_list = ['05/30/SID2_20230530T100326-20230530T100925.cdf',
                  '05/30/SID2_20230530T100927-20230530T100937.cdf',
                  '06/01/SID2_20230601T120759-20230601T120857.cdf',
                  '06/01/SID2_20230601T121435-20230601T121533.cdf',
                  '06/01/SID2_20230601T122138-20230601T122236.cdf',
                  '06/01/SID2_20230601T122707-20230601T122805.cdf',
                  '07/12/SID2_20230712T090434-20230712T093848.cdf',
                  '07/12/SID2_20230712T093942-20230712T101355.cdf',
                  '07/12/SID2_20230712T101449-20230712T104147.cdf',
                  '07/12/SID2_20230712T104149-20230712T232406.cdf',
                  '07/12/SID2_20230712T232408-20230712T235156.cdf',
                  '07/12/SID2_20230712T235158-20230713T001854.cdf',
                  '07/13/SID2_20230713T001856-20230713T004644.cdf',
                  '07/13/SID2_20230713T004648-20230713T011342.cdf',
                  '07/13/SID2_20230713T011436-20230713T014134.cdf',
                  '07/13/SID2_20230713T014136-20230713T020924.cdf',
                  '07/13/SID2_20230713T020928-20230713T023718.cdf',
                  '07/13/SID2_20230713T023720-20230713T030416.cdf',
                  '07/13/SID2_20230713T030510-20230713T033208.cdf',
                  '07/13/SID2_20230713T033210-20230713T040000.cdf',
                  '07/13/SID2_20230713T040002-20230713T042751.cdf',
                  '07/13/SID2_20230713T042753-20230713T045449.cdf',
                  '07/13/SID2_20230713T045543-20230713T050917.cdf',
                 ]
"""

In [ ]:
# *** Flight data: Ver.2 ***
# 20240126
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2024/'        # CDF data folder

data_name_list = ['01/25/JUICE_L1a_RPWI-HF-SID2_20240125T112327_V01.cdf',      # 128
                  #'01/25/JUICE_L1a_RPWI-HF-SID2_20240125T152238_V01.cdf',     # 512
                 ]
"""
"""

In [ ]:
# *** Group read
"""
import glob
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name = 'SID02*.cdf'
cdf_file = data_dir + data_name
data_name_list = glob.glob(cdf_file)
num_list = len(data_name_list)
data_name_list.sort()
for i in range(num_list):
    data_name_list[i] = os.path.split(data_name_list[i])[1]
"""

# get CDF data

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)

for i in range(num_list):
    data_name = data_name_list[i]
    cdf_file = data_dir + data_name
    print(i, cdf_file)

    cdf = pycdf.CDF(cdf_file)
    data1 = juice_sid2.hf_sid2_read(cdf)

    if i==0:
        data = data1
        print(data.Eu_i.shape)
    else:
        data = juice_sid2.hf_sid2_add(data, data1)
        print(data.Eu_i.shape)

print("path name:", data_name);  data_name = os.path.split(data_name)[1];  print("data name:", data_name)

In [ ]:
data = juice_sid2.hf_sid2_proc(data)
date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
print(str_date)

n_time0 = data.Eu_i.shape[0]
n_freq0 = data.Eu_i.shape[1]
n_samp0 = data.Eu_i.shape[2]
f_min0 = data.frequency[0][0][0]
f_max0 = (max(np.ravel(data.frequency)))

In [ ]:
# Mode 
N_ch0 = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
print("Ch:", N_ch0, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
print("Num-samples:", n_time0, "   Num-Frequency:", n_freq0, "   Length:", n_samp0)
print("Frequency, width, step (kHz):", f_min0,  "-", f_max0, data.freq_width[0][0][0], data.freq_step[0][0][0])
print("Time-length:", data.time[0][0][n_samp0-1], "sec in 1-sweep")
# print(data.epoch)

# Raw data

In [ ]:
fig = plt.figure(figsize=(14, 10))
ax1 = fig.add_subplot(5, 1, 1);  ax2 = fig.add_subplot(5, 1, 2);  ax3 = fig.add_subplot(5, 1, 3)
ax4 = fig.add_subplot(5, 1, 4);  ax5 = fig.add_subplot(5, 1, 5)

ax1.plot(np.ravel(data.Eu_i[:][:]), '-r', linewidth=.5, label='Eu_i'); ax1.plot(np.ravel(data.Eu_q[:][:]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(data.Ev_i[:][:]), '-r', linewidth=.5, label='Ev_i'); ax2.plot(np.ravel(data.Ev_q[:][:]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(data.Ew_i[:][:]), '-r', linewidth=.5, label='Ew_i'); ax3.plot(np.ravel(data.Ew_q[:][:]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency),  '-b', linewidth=.5, label='Frequency');
ax4.plot(np.ravel(data.freq_step*10),  '-g', linewidth=0.8, label='step*10')
ax4.plot(np.ravel(data.freq_width*10), ':b', linewidth=1.0, label='width*10')
ax4.plot(np.ravel(data.sweep_start)*data.frequency[0][-1][0], '-r', linewidth=.5, label='Sweep Start');
ax5.plot(np.ravel(data.epoch[:]), '.')
ax1.set_ylabel('Eu');  ax2.set_ylabel('Ev');  ax3.set_ylabel('Ew');  ax4.set_ylabel('Frequency [kHz]');  ax5.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF RAW (SID-2)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, len(np.ravel(data.Eu_i[:][:]))];  print(xlim)
ax1.set_xlim(xlim);  ax3.set_xlim(xlim);  ax5.set_xlim(xlim);  ax7.set_xlim(xlim)
"""
# range: Y-axis
"""
ylim=[-10**(p_max0-3.5), 10**(p_max0-3.5)]
ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)
"""
ylim=[f_min0, f_max0]; ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

## RAW - First 2 Sweeps

In [ ]:
n_sweep0 = 0;    n_sweep1 = n_sweep0 + 2
print("[specific sweep]  SWEEP:", n_sweep0, n_sweep1, "in total of", n_time0)
f_min = f_min0;  f_max = f_max0

fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(np.ravel(data.Eu_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Eu_i'); 
ax1.plot(np.ravel(data.Eu_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(data.Ev_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Ev_i'); 
ax2.plot(np.ravel(data.Ev_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(data.Ew_i[n_sweep0:n_sweep1]), '-r', linewidth=.5, label='Ew_i'); 
ax3.plot(np.ravel(data.Ew_q[n_sweep0:n_sweep1]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency[n_sweep0:n_sweep1]),  '-b', linewidth=.5, label='Frequency');
ax4.plot(np.ravel(data.sweep_start[n_sweep0:n_sweep1])*data.frequency[n_sweep0][-1][0], '-r', label='Sweep Start');
ax1.set_ylabel('Eu');  ax2.set_ylabel('Ev');  ax3.set_ylabel('Ew');  ax4.set_ylabel('Frequency [kHz]');  
date1 = data.epoch[n_sweep0];  date1 = date1.strftime('%Y/%m/%d %R:%S');  ax4.set_xlabel(date1)
#
title_label = '[JUICE/RPWI HF RAW (SID-2)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[len(np.ravel(data.Eu_i[n_sweep0:n_sweep1]))//2 - 32, len(np.ravel(data.Eu_i[n_sweep0:n_sweep1]))//2 + 32];  print(xlim)
ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
"""
# range: Y-axis
"""
ylim=[-10**(p_max0), 10**(p_max0)]; ax1.set_ylim(ylim);  ax2.set_ylim(ylim);  ax3.set_ylim(ylim)
"""
ylim=[f_min, f_max]; ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-sweep.png'
    fig.savefig(png_fname)

## Raw - First 3 steps

In [ ]:
n_sweep = n_time0//2;  n_step0 = 0;  n_step1 = n_step0 + 3
print("[specific sweep]  SWEEP:", n_sweep, "in total of", n_time0,  "[Steps]", n_step0, "-", n_step1)

fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(np.ravel(data.Eu_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Eu_i')
ax1.plot(np.ravel(data.Eu_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Eu_q')
ax2.plot(np.ravel(data.Ev_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Ev_i')
ax2.plot(np.ravel(data.Ev_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Ev_q')
ax3.plot(np.ravel(data.Ew_i[n_sweep][n_step0:n_step1]), '-r', linewidth=.5, label='Ew_i')
ax3.plot(np.ravel(data.Ew_q[n_sweep][n_step0:n_step1]), ':g', linewidth=.5, label='Ew_q')
ax4.plot(np.ravel(data.frequency[n_sweep][n_step0:n_step1]),  '-b', linewidth=.5, label='Frequency')
ax4.plot(np.ravel(data.sweep_start[n_sweep][n_step0:n_step1])*data.frequency[n_sweep][n_step1][0], '-r', label='Sweep Start')
ax1.set_ylabel('Eu');  ax2.set_ylabel('Ev');  ax3.set_ylabel('Ew');  ax4.set_ylabel('Frequency [kHz]');  
date1 = data.epoch[n_sweep];  date1 = date1.strftime('%Y/%m/%d %R:%S');  ax4.set_xlabel(date1)
#
title_label = '[JUICE/RPWI HF RAW (SID-2)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, len(np.ravel(data.Eu_i[:][:]))];  print(xlim)
ax1.set_xlim(xlim);  ax3.set_xlim(xlim);  ax5.set_xlim(xlim);  ax7.set_xlim(xlim)
"""
# range: Y-axis
"""
ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)
"""
#ylim=[f_min, f_max]; ax4.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-sweep.png'
    fig.savefig(png_fname)

# Get Spectrum

In [ ]:
# Spec mode: high resolution  
spec = juice_sid2.hf_sid2_getspec(data, cal_mode)
Epoch_1d = spec.epoch.tolist();   n_time0 = spec.EE.shape[0];  num_1d = np.arange(n_time0)
freq_1d = spec.freq[n_time0//2];  n_freq0 = freq_1d.shape[0]

EE_2d   = spec.EE.transpose();    EE_med  = np.median(spec.EE, axis=0)

power_str = juice_cdf.power_label(band_mode, unit_mode)

print(power_str, "EE:", spec.EE.shape, freq_1d[0], "-", freq_1d[-1], "kHz  (df =", freq_1d[1] - freq_1d[0], "kHz)", n_freq0 )

# Spectrum

In [ ]:
# Sweep_num
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
n_sweep1 = n_time0//4;  n_sweep2 = 3*n_time0//4;  n_sweep3 = n_time0-1
# n_sweep1 = 50; n_sweep2 = 1600;  n_sweep3 = 2000
# Y-range
p_min = p_min0;   p_max = p_max0
# X-range
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])

fig = plt.figure(figsize=(14, 8));  ax1 = fig.add_subplot(1, 1, 1)

ax1.plot(freq_1d, spec.EE[n_sweep1], '-r', linewidth=.9, label='EE 1st')
ax1.plot(freq_1d, spec.EE[n_sweep2], '-g', linewidth=.4, label='EE mid')
ax1.plot(freq_1d, spec.EE[n_sweep3], '-b', linewidth=.2, label='EE last')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')

# Label
ax1.set_ylabel('EE')
#
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
# xlim=[8150, 8250]
ax1.set_xlim(xlim)

# range: Y-axis
ylim=[10**(p_min), 10**(p_max)]
ax1.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

# Peak
print(f"[First - {n_sweep1}]"); peak_E = np.ravel(spec.EE[n_sweep1]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
print(f"[Mid - {n_sweep2}]");   peak_E = np.ravel(spec.EE[n_sweep2]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")
print(f"[Last - {n_sweep3}]");  peak_E = np.ravel(spec.EE[n_sweep3]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq_1d[peak_f]), "kHz")


# Frequency - Time diagram

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
# f_min = 41900;      f_max = 42100
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max;  print(data.frequency[0][0][0], data.frequency[0][8][0])
# Z-range
p_min  = p_min0;     p_max  = p_max0
#p_min  = p_min0+1.5;     p_max  = p_max0-2.5

if time_mode == 1:
    fig2d = plt.figure(figsize=[14,12]);  ax1 = fig2d.add_subplot(2, 1, 1);  ax2 = fig2d.add_subplot(2, 1, 2);  ax2.set_xlabel(str_date)
else:
    fig2d = plt.figure(figsize=[14,8]);   ax1 = fig2d.add_subplot(1, 1, 1);                                     ax1.set_xlabel(str_date)

# X-axis
ax1.set_title('EE')

# Y-axis
ax1.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]')

# Plot
print(num_1d.shape, freq_1d.shape, EE_2d.shape)
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EE_2d,   norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.plot(np.ravel(spec.epoch[:]), '.')
    pp1 = fig2d.colorbar(p1, ax=ax2, orientation="vertical"); pp1.set_label(power_str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d,   norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)

# range: X-axis
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax1.set_xlim(xlim); ax2.set_xlim(xlim)

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)

# Mask Write / Read for SID-3

In [ ]:
asw_ver = 2;  freq_1d_sid3, f_step_sid3, f_width_sid3 = juice_cdf._frequency_sid3(asw_ver)

# Frequency of SID-3 from SID-2  --  [tbl_freq_to_sid]  0 - start ch, 1 - end ch,  2 - width ch
n_freq_sid3 = len(freq_1d_sid3)
tbl_sid2_to_sid3 = juice_cdf._frequency_sid2_to_data(freq_1d_sid3, f_step_sid3, freq_1d)

# Frequency of SID-3 from MASK   --  [tbl_freq_to_sid]  0 - start ch, 1 - end ch,  2 - width ch
n_mask = 19392;   freq_1d_msk = np.zeros(n_mask)
for i in range(n_mask):
    freq_1d_msk[i] = 80.0 + 2.3125 * i
n_freq_msk = len(freq_1d_msk)
tbl_msk_to_sid3 = juice_cdf._frequency_sid2_to_data(freq_1d_sid3, f_step_sid3, freq_1d_msk)

In [ ]:
# init
spec_sid3_EE      = np.zeros(n_freq_sid3)
spec_sid3_EE_mean = np.zeros(n_freq_sid3)
spec_sid3_EE_med  = np.zeros(n_freq_sid3)
#
Diff_sid3         = 1.0
mask_list0 = np.zeros(n_freq0);  mask_list1 = np.zeros(n_freq0);  mask_list2 = np.zeros(n_freq0);  mask_list3 = np.zeros(n_freq0)
EE_med_sid3       = np.zeros(n_freq0)   # threshold
EE_med_sid3_diff  = np.zeros(n_freq0)

# SID-3 from SID-2
for j in range(n_freq_sid3):
    # SID-3
    j0 = tbl_sid2_to_sid3[j][0];  j1 = tbl_sid2_to_sid3[j][1]
    spec_sid3_EE[j]      = np.sum(   EE_med[j0:j1+1])
    spec_sid3_EE_med[j]  = np.median(EE_med[j0:j1+1])
    spec_sid3_EE_mean[j] = np.mean(  EE_med[j0:j1+1])
    # 
    EE_med_sid3[j0:j1+1] = spec_sid3_EE_med[j]
    mask_list0[j0:j1+1]  = j0                       # Ch start
    mask_list1[j0:j1+1]  = j1                       # Ch end
    mask_list2[j0:j1+1]  = tbl_sid2_to_sid3[j][2]   # Ch width
    mask_list3[j0:j1+1]  = j                        # Ch_SID03 
EE_med_sid3[j1:n_freq0]  = spec_sid3_EE_med[255]
#
EE_dif_med = 10 * np.log10(EE_med) - 10 * np.log10(EE_med_sid3 * 10**(Diff_sid3/10.))

In [ ]:
# init: MASK
mask_dir  = './lib/';  mask_name = 'hf_sid3_mask.csv';  mask_file = mask_dir + mask_name
mask_list = np.zeros(n_mask * 3);  mask_list  = mask_list.reshape(n_mask, 3)

# Write Mask
if filter_mode > 0:
    for i in range(n_mask):
        mask_list[i, 0] = freq_1d_msk[i]           # f_min
        mask_list[i, 1] = freq_1d_msk[i] + 2.3125  # f_max
        mask_list[i, 2] = 0                        # mask

    for i in range(n_freq0):
        f0 = freq_1d[ i ] 
        if (filter_mode2 > 0):
            i0 = np.int16(round((f0 - 80.0) / 2.3125))
            if i0<n_mask and EE_dif_med[i] >= Diff_sid3:
                mask_list[i0, 2] = 1                   # mask
        else: 
            i0 = np.int16(np.floor((f0 - 80.0) / 2.3125))
            if i0<n_mask and EE_dif_med[i] >= Diff_sid3:
                mask_list[i0, 2] = 1                   # mask
            i1 = np.int16(np.ceil ((f0 - 80.0) / 2.3125))
            if i1<n_mask and EE_dif_med[i] >= Diff_sid3:
                mask_list[i1, 2] = 1                   # mask

    with open(mask_file, 'w') as f:
        writer = csv.writer(f)
        for i in range(n_mask):
            writer.writerow([ i+1, mask_list[i, 0], mask_list[i, 1], np.int16(mask_list[i, 2]) ])

# Read Mask
with open(mask_file, 'r') as f:
    reader = csv.reader(f)
    i = 0
    for row in reader:
        mask_list[i, 0] = row[1]
        mask_list[i, 1] = row[2]
        mask_list[i, 2] = row[3]
        i = i+1

# Mask Apply to SID2 for SID3 emulation

In [ ]:
EE_med_msk = copy.deepcopy(EE_med)
EE_msk = np.zeros(n_mask)
spec_sid3_EE_msk      = np.zeros(n_freq_sid3)
spec_sid3_EE_mean_msk = np.zeros(n_freq_sid3)

# masking to SID-2
for i in range(n_freq0):
    f0 = freq_1d[ i ] 
    if (filter_mode2 > 0):
        i0 = np.int16(round((f0 - 80.0) / 2.3125))
        if i0<n_mask and mask_list[i0, 2] > 0:
            EE_med_msk[i] = 0
    else:
        i0 = np.int16(np.floor((f0 - 80.0) / 2.3125))
        if i0<n_mask and mask_list[i0, 2] > 0:
            EE_med_msk[i] = 0
        i1 = np.int16(np.ceil ((f0 - 80.0) / 2.3125))
        if i1<n_mask and mask_list[i1, 2] > 0:
            EE_med_msk[i] = 0

for i in range(n_mask):
    EE_msk[i] = mask_list[i, 2]

# SID-3 from masked SID-2 
for j in range(n_freq_sid3):
    # SID-3
    j0 = tbl_sid2_to_sid3[j][0];  j1 = tbl_sid2_to_sid3[j][1]
    spec_sid3_EE_msk[j] = np.sum(EE_med_msk[j0:j1+1])

    num = 0
    for i in range(j0, j1+1):
        if EE_med_msk[i]>0:
            num = num+1
    spec_sid3_EE_mean_msk[j] = spec_sid3_EE_msk[j] / num

# SID-3 from masked SID-2 : peaks
from scipy.signal import find_peaks
peaks,_ = find_peaks(EE_med_msk, height=3.0)

### SID-3 Frequency table with MASK

In [ ]:
# SID-3 Frequeny table write
freq_dir  = './lib/';  freq_name = 'hf_sid3_freq.csv';  freq_file = freq_dir + freq_name
with open(freq_file, 'w') as f:
    writer = csv.writer(f)

    for i in range(n_freq_sid3):
        i0 = tbl_msk_to_sid3[i][0];  i1 = tbl_msk_to_sid3[i][1]
        num = 0
        for j in range(i0, i1+1):
            if EE_msk[j]==0:
                num = num +1
        # print(i, freq_1d_sid3[i], tbl_msk_to_sid3[i][2], num, num/tbl_msk_to_sid3[i][2]*100, i0, i1, freq_1d_msk[i0], freq_1d_msk[i1], )

        writer.writerow([ i+1, freq_1d_sid3[i], tbl_msk_to_sid3[i][2], num, num/tbl_msk_to_sid3[i][2]*100, freq_1d_msk[i0], freq_1d_msk[i1] ])

### SID3 spectrum with Mask

In [ ]:
# Range
p_min = p_min0+1;  p_max = p_max0
f_min = f_min0;    f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

fig = plt.figure(figsize=(14, 12));  ax1 = fig.add_subplot(1, 1, 1)
ax1.plot(freq_1d,      EE_med_msk,           '-y', linewidth=4,  label='EE_msk (SID2)')
ax1.plot(freq_1d,      EE_med,               ':k', linewidth=.4,  label='EE (SID2)')
ax1.plot(freq_1d,      EE_med_sid3,          '-k', linewidth=.8,  label='EE_sid3 (SID2)')
# ax1.plot(freq_1d_sid3, spec_sid3_EE,         ':r', linewidth=1.2, label='EE(SID-3)')
# ax1.plot(freq_1d_sid3, spec_sid3_EE_msk,     '-r', linewidth=1.2, label='msked EE (SID3)')
ax1.plot(freq_1d_msk,  EE_msk*20,            '-c', linewidth=.3,  label='mask pattern')
ax1.plot(freq_1d_sid3, spec_sid3_EE_mean_msk,'-g', linewidth=3.0, label='msked EE_mean (SID3)')
ax1.plot(freq_1d_sid3, spec_sid3_EE_mean,    ':b', linewidth=1.2, label='raw EE_mean (SID3)')
ax1.plot(freq_1d_sid3, spec_sid3_EE_med,     '-r', linewidth=.8,  label='raw EE_med(SID-3)')
# ax1.plot(freq_1d,      EE_dif_med*100,   '.g', linewidth=.2,  label='Diff')
ax1.plot(freq_1d[peaks], EE_med_msk[peaks],  '.r',                label='EE_msk (SID2) - peak')

ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')
    
# range
xlim=[f_min, f_max];              ax1.set_xlim(xlim)
ylim=[10**(p_min), 10**(p_max)];  ax1.set_ylim(ylim)

# Label
ax1.set_ylabel('EE '+power_str)
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime('1st: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime('Mid: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime('Fin: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date);   ax1.legend(loc='upper right', fontsize=8);  

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-sid3emu.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-sid3emu-log.png'
    fig.savefig(png_fname)

In [ ]:
# Y-range
p_min = p_min0+1;   p_max = p_max0
# X-range
f_min = 0;          f_max = 5000
# f_min = 4700;       f_max = 4900

fig = plt.figure(figsize=(14, 40))
ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4)
ax5 = fig.add_subplot(9, 1, 5);  ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8)
ax9 = fig.add_subplot(9, 1, 9)

ax1.plot(freq_1d,      EE_med_msk,           '-y', linewidth=4,  label='EE_msk (SID2)')
ax1.plot(freq_1d,      EE_med,               ':k', linewidth=.4,  label='EE (SID2)')
ax1.plot(freq_1d,      EE_med_sid3,          '-k', linewidth=.8,  label='EE_sid3 (SID2)')
ax1.plot(freq_1d_msk,  EE_msk*20,            '-c', linewidth=.3,  label='mask pattern')
ax1.plot(freq_1d_sid3, spec_sid3_EE_mean_msk,'-g', linewidth=3.0, label='msked EE_mean (SID3)')
ax1.plot(freq_1d_sid3, spec_sid3_EE_mean,    ':b', linewidth=1.2, label='raw EE_mean (SID3)')
ax1.plot(freq_1d_sid3, spec_sid3_EE_med,     '-r', linewidth=.8,  label='raw EE_med(SID-3)')
ax1.plot(freq_1d[peaks], EE_med_msk[peaks],  '.r',                label='EE_msk (SID2) - peak')
#
ax2.plot(freq_1d,      EE_med_msk,           '-y', linewidth=4,  label='EE_msk (SID2)')
ax2.plot(freq_1d,      EE_med,               ':k', linewidth=.4,  label='EE (SID2)')
ax2.plot(freq_1d,      EE_med_sid3,          '-k', linewidth=.8,  label='EE_sid3 (SID2)')
ax2.plot(freq_1d_msk,  EE_msk*20,            '-c', linewidth=.3,  label='mask pattern')
ax2.plot(freq_1d_sid3, spec_sid3_EE_mean_msk,'-g', linewidth=3.0, label='msked EE_mean (SID3)')
ax2.plot(freq_1d_sid3, spec_sid3_EE_mean,    ':b', linewidth=1.2, label='raw EE_mean (SID3)')
ax2.plot(freq_1d_sid3, spec_sid3_EE_med,     '-r', linewidth=.8,  label='raw EE_med(SID-3)')
ax2.plot(freq_1d[peaks], EE_med_msk[peaks],  '.r',                label='EE_msk (SID2) - peak')
#
ax3.plot(freq_1d,      EE_med_msk,           '-y', linewidth=4,  label='EE_msk (SID2)')
ax3.plot(freq_1d,      EE_med,               ':k', linewidth=.4,  label='EE (SID2)')
ax3.plot(freq_1d,      EE_med_sid3,          '-k', linewidth=.8,  label='EE_sid3 (SID2)')
ax3.plot(freq_1d_msk,  EE_msk*20,            '-c', linewidth=.3,  label='mask pattern')
ax3.plot(freq_1d_sid3, spec_sid3_EE_mean_msk,'-g', linewidth=3.0, label='msked EE_mean (SID3)')
ax3.plot(freq_1d_sid3, spec_sid3_EE_mean,    ':b', linewidth=1.2, label='raw EE_mean (SID3)')
ax3.plot(freq_1d_sid3, spec_sid3_EE_med,     '-r', linewidth=.8,  label='raw EE_med(SID-3)')
ax3.plot(freq_1d[peaks], EE_med_msk[peaks],  '.r',                label='EE_msk (SID2) - peak')
#
ax4.plot(freq_1d,      EE_med_msk,           '-y', linewidth=4,  label='EE_msk (SID2)')
ax4.plot(freq_1d,      EE_med,               ':k', linewidth=.4,  label='EE (SID2)')
ax4.plot(freq_1d,      EE_med_sid3,          '-k', linewidth=.8,  label='EE_sid3 (SID2)')
ax4.plot(freq_1d_msk,  EE_msk*20,            '-c', linewidth=.3,  label='mask pattern')
ax4.plot(freq_1d_sid3, spec_sid3_EE_mean_msk,'-g', linewidth=3.0, label='msked EE_mean (SID3)')
ax4.plot(freq_1d_sid3, spec_sid3_EE_mean,    ':b', linewidth=1.2, label='raw EE_mean (SID3)')
ax4.plot(freq_1d_sid3, spec_sid3_EE_med,     '-r', linewidth=.8,  label='raw EE_med(SID-3)')
ax4.plot(freq_1d[peaks], EE_med_msk[peaks],  '.r',                label='EE_msk (SID2) - peak')
#
ax5.plot(freq_1d,      EE_med_msk,           '-y', linewidth=4,  label='EE_msk (SID2)')
ax5.plot(freq_1d,      EE_med,               ':k', linewidth=.4,  label='EE (SID2)')
ax5.plot(freq_1d,      EE_med_sid3,          '-k', linewidth=.8,  label='EE_sid3 (SID2)')
ax5.plot(freq_1d_msk,  EE_msk*20,            '-c', linewidth=.3,  label='mask pattern')
ax5.plot(freq_1d_sid3, spec_sid3_EE_mean_msk,'-g', linewidth=3.0, label='msked EE_mean (SID3)')
ax5.plot(freq_1d_sid3, spec_sid3_EE_mean,    ':b', linewidth=1.2, label='raw EE_mean (SID3)')
ax5.plot(freq_1d_sid3, spec_sid3_EE_med,     '-r', linewidth=.8,  label='raw EE_med(SID-3)')
ax5.plot(freq_1d[peaks], EE_med_msk[peaks],  '.r',                label='EE_msk (SID2) - peak')
#
ax6.plot(freq_1d,      EE_med_msk,           '-y', linewidth=4,  label='EE_msk (SID2)')
ax6.plot(freq_1d,      EE_med,               ':k', linewidth=.4,  label='EE (SID2)')
ax6.plot(freq_1d,      EE_med_sid3,          '-k', linewidth=.8,  label='EE_sid3 (SID2)')
ax6.plot(freq_1d_msk,  EE_msk*20,            '-c', linewidth=.3,  label='mask pattern')
ax6.plot(freq_1d_sid3, spec_sid3_EE_mean_msk,'-g', linewidth=3.0, label='msked EE_mean (SID3)')
ax6.plot(freq_1d_sid3, spec_sid3_EE_mean,    ':b', linewidth=1.2, label='raw EE_mean (SID3)')
ax6.plot(freq_1d_sid3, spec_sid3_EE_med,     '-r', linewidth=.8,  label='raw EE_med(SID-3)')
ax6.plot(freq_1d[peaks], EE_med_msk[peaks],  '.r',                label='EE_msk (SID2) - peak')
#
ax7.plot(freq_1d,      EE_med_msk,           '-y', linewidth=4,  label='EE_msk (SID2)')
ax7.plot(freq_1d,      EE_med,               ':k', linewidth=.4,  label='EE (SID2)')
ax7.plot(freq_1d,      EE_med_sid3,          '-k', linewidth=.8,  label='EE_sid3 (SID2)')
ax7.plot(freq_1d_msk,  EE_msk*20,            '-c', linewidth=.3,  label='mask pattern')
ax7.plot(freq_1d_sid3, spec_sid3_EE_mean_msk,'-g', linewidth=3.0, label='msked EE_mean (SID3)')
ax7.plot(freq_1d_sid3, spec_sid3_EE_mean,    ':b', linewidth=1.2, label='raw EE_mean (SID3)')
ax7.plot(freq_1d_sid3, spec_sid3_EE_med,     '-r', linewidth=.8,  label='raw EE_med(SID-3)')
ax7.plot(freq_1d[peaks], EE_med_msk[peaks],  '.r',                label='EE_msk (SID2) - peak')
#
ax8.plot(freq_1d,      EE_med_msk,           '-y', linewidth=4,  label='EE_msk (SID2)')
ax8.plot(freq_1d,      EE_med,               ':k', linewidth=.4,  label='EE (SID2)')
ax8.plot(freq_1d,      EE_med_sid3,          '-k', linewidth=.8,  label='EE_sid3 (SID2)')
ax8.plot(freq_1d_msk,  EE_msk*20,            '-c', linewidth=.3,  label='mask pattern')
ax8.plot(freq_1d_sid3, spec_sid3_EE_mean_msk,'-g', linewidth=3.0, label='msked EE_mean (SID3)')
ax8.plot(freq_1d_sid3, spec_sid3_EE_mean,    ':b', linewidth=1.2, label='raw EE_mean (SID3)')
ax8.plot(freq_1d_sid3, spec_sid3_EE_med,     '-r', linewidth=.8,  label='raw EE_med(SID-3)')
ax8.plot(freq_1d[peaks], EE_med_msk[peaks],  '.r',                label='EE_msk (SID2) - peak')
#
ax9.plot(freq_1d,      EE_med_msk,           '-y', linewidth=4,  label='EE_msk (SID2)')
ax9.plot(freq_1d,      EE_med,               ':k', linewidth=.4,  label='EE (SID2)')
ax9.plot(freq_1d,      EE_med_sid3,          '-k', linewidth=.8,  label='EE_sid3 (SID2)')
ax9.plot(freq_1d_msk,  EE_msk*20,            '-c', linewidth=.3,  label='mask pattern')
ax9.plot(freq_1d_sid3, spec_sid3_EE_mean_msk,'-g', linewidth=3.0, label='msked EE_mean (SID3)')
ax9.plot(freq_1d_sid3, spec_sid3_EE_mean,    ':b', linewidth=1.2, label='raw EE_mean (SID3)')
ax9.plot(freq_1d_sid3, spec_sid3_EE_med,     '-r', linewidth=.8,  label='raw EE_med(SID-3)')
ax9.plot(freq_1d[peaks], EE_med_msk[peaks],  '.r',                label='EE_msk (SID2) - peak')
#
 
ax1.set_yscale('log');  ax2.set_yscale('log');  ax3.set_yscale('log');  ax4.set_yscale('log');  ax5.set_yscale('log')
ax6.set_yscale('log');  ax7.set_yscale('log');  ax8.set_yscale('log');  ax9.set_yscale('log')

# Label
ax1.set_ylabel(power_str); ax2.set_ylabel(power_str); ax3.set_ylabel(power_str); ax4.set_ylabel(power_str); ax5.set_ylabel(power_str)
ax6.set_ylabel(power_str); ax7.set_ylabel(power_str); ax8.set_ylabel(power_str); ax9.set_ylabel(power_str)
ax9.set_xlabel('Frequency [kHz]')
#
date1 = Epoch_1d[n_sweep1];  date1 = date1.strftime(f'1st {n_sweep1}: %Y/%m/%d %R:%S   ')
date2 = Epoch_1d[n_sweep2];  date2 = date2.strftime(f'Mid {n_sweep2}: %Y/%m/%d %R:%S   ')
date3 = Epoch_1d[n_sweep3];  date3 = date3.strftime(f'Fin {n_sweep3}: %Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)

ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);  ax6.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8);  ax8.legend(loc='upper right', fontsize=8);  ax9.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min,       f_max];        ax1.set_xlim(xlim); 
xlim=[f_min+5000,  f_max+5000];   ax2.set_xlim(xlim);  
xlim=[f_min+10000, f_max+10000];  ax3.set_xlim(xlim); 
xlim=[f_min+15000, f_max+15000];  ax4.set_xlim(xlim)
xlim=[f_min+20000, f_max+20000];  ax5.set_xlim(xlim)
xlim=[f_min+25000, f_max+25000];  ax6.set_xlim(xlim)
xlim=[f_min+30000, f_max+30000];  ax7.set_xlim(xlim)
xlim=[f_min+35000, f_max+35000];  ax8.set_xlim(xlim)
xlim=[f_min+40000, f_max+40000];  ax9.set_xlim(xlim)

# range: Y-axis
ylim=[10**p_min, 10**p_max]
# ylim=[1, 50]
ax1.set_ylim(ylim); ax2.set_ylim(ylim);  ax3.set_ylim(ylim); ax4.set_ylim(ylim); ax5.set_ylim(ylim); ax6.set_ylim(ylim);  
ax7.set_ylim(ylim); ax8.set_ylim(ylim);  ax9.set_ylim(ylim)

# MASK filter